In [ ]:
import os
import pandas as pd
import numpy as np

In [1]:
annot = pd.read_csv('QPW/TikTok Airports Data - Gray Man.csv')

annot

In [2]:
# Head annotations
annot['row_y'] = 0
annot['col_x'] = 105
annot['width']=45
annot['height']=45

annot

filename                                            label  \
0     gray man demo.png                          Left and right shoulder   
1    gray man demo1.png                 Left shoulder and left upper leg   
2    gray man demo2.png                                             Back   
3    gray man demo3.png                        Right front and back head   
4    gray man demo4.png  Right stomach and left upper leg and right foot   
..                  ...                                              ...   
95  gray man demo95.png   back of neck and right upper leg and left knee   
96  gray man demo96.png                            left buttock and back   
97  gray man demo97.png            right armpit and left armpit and back   
98  gray man demo98.png            right arm and left arm and right foot   
99  gray man demo99.png              back of neck and back and left foot   

    row_y  col_x  width  height  
0       0    105     45      45  
1       0    105     45      45  
2       0    105     45      45  
3       0    105     45      45  
4       0    105     45      45  
..    ...    ...    ...     ...  
95      0    105     45      45  
96      0    105     45      45  
97      0    105     45      45  
98      0    105     45      45  
99      0    105     45      45  

[100 rows x 6 columns]

In [3]:
annot['label'] = annot['label'].apply(lambda x: x.replace('foot', 'leg'))
annot['label'] = annot['label'].apply(lambda x: x.lower())
annot['label'] = annot['label'].apply(lambda x: x.replace('back of', ''))
annot['label2'] = annot['label'].copy()
annot['label'] = annot['label2'].apply(lambda x: 'head' in x)

In [4]:
annot2 = annot.copy()
annot2['row_y'] = 92
annot2['col_x'] = 60
annot2['width'] = 31
annot2['height'] = 110
print(annot2)
annot2['label'] = annot2['label2'].apply(lambda x: 'right hand' in x)
annot = pd.concat([annot,annot2])

               filename  label  row_y  col_x  width  height  \
0     gray man demo.png  False     92     60     31     110   
1    gray man demo1.png  False     92     60     31     110   
2    gray man demo2.png  False     92     60     31     110   
3    gray man demo3.png   True     92     60     31     110   
4    gray man demo4.png  False     92     60     31     110   
..                  ...    ...    ...    ...    ...     ...   
95  gray man demo95.png  False     92     60     31     110   
96  gray man demo96.png  False     92     60     31     110   
97  gray man demo97.png  False     92     60     31     110   
98  gray man demo98.png  False     92     60     31     110   
99  gray man demo99.png  False     92     60     31     110   

                                            label2  
0                          left and right shoulder  
1                 left shoulder and left upper leg  
2                                             back  
3                        right 

In [5]:
annot.to_parquet('gray_man_demo_2/vl_annotations_images.parquet')

In [6]:
annot.to_parquet('gray_man_demo_2/vl_annotations_objects.parquet')

In [9]:
# import fastdup
# fd = fastdup.create(input_dir='gray_man_demo_2/', work_dir='out')
# fd.run(annotations=annot.copy(),ccthreshold=0.998,overwrite=1)

ModuleNotFoundError: No module named 'fastdup'

In [10]:
# fd.vis.component_gallery()

NameError: name 'fd' is not defined

In [11]:
# fd.vis.outliers_gallery()

NameError: name 'fd' is not defined

# Create annotations demo using provided xls file

In [ ]:
!pip install xlrd pyarrow

In [29]:
import pandas as pd
import numpy as np

def create_full_label(df):
    side_col = df.iloc[:, 2]
    part_col = df['Location']
    
    # Create the 'full_label' column using np.where
    df['full_label'] = np.where(side_col.isna(), part_col, side_col + ' ' + part_col)
    
    return df

def process_airports_df(df, data_dir, body_part, save_parquet = True):
    df_copy = df.copy()
    df_copy = create_full_label(df_copy)
    
    # Check if 'body_part' is in 'full_label' column and store the result in 'label' column
    df_copy['label'] = df_copy['full_label'].apply(lambda x: body_part in x.lower())
    
    # Get the list of image filenames in 'data_dir'
    image_list = os.listdir(data_dir)
    
    # Find matching filenames for each 'File' and store the result in 'filename' column
    df_copy['filename'] = df_copy['File'].apply(lambda x: [o for o in image_list if str(x) in o])
    
    # Explode the 'filename' column to create multiple rows when there are multiple filenames
    df_copy = df_copy.explode('filename')
    df_copy = df_copy[['filename', 'label', 'File', 'Location', 'full_label']]
    
    if save_parquet:
        save_path = os.path.join(data_dir, 'vl_annotations_objects.parquet')
        df_copy.to_parquet(save_path)
        print(f"Saved annotations to {save_path} -> Use ./run_profiler.sh {data_dir}")
    
    return df_copy

In [26]:
data_dir = '/Users/amirm/vl_data/tiktok_airports/QPW/'
xls_qpw = pd.read_excel(os.path.join(data_dir, 'Book1.xlsx'), sheet_name=None)
xls_qpw.keys()

dict_keys(['QPW', 'QPS201', 'Sheet3'])

In [27]:
xls_qpw['QPW']

File   Location Unnamed: 2
0  120853     forarm      Right
1  100146       Head        NaN
2  100230  Lower Leg      Right
3  131930       Back        NaN
4  134505      Belly        NaN
5  105623       Neck        NaN
6  103401       Head        NaN
7  102925       Head        NaN

In [30]:
# Example usage:
body_part = 'head'
qpw_df = xls_qpw['QPW']
qpw_df_head = process_airports_df(qpw_df, data_dir, body_part)
qpw_df_head

Saved annotations to /Users/amirm/vl_data/tiktok_airports/QPW/vl_annotations_objects.parquet -> Use ./run_profiler.sh /Users/amirm/vl_data/tiktok_airports/QPW/


filename  label    File   Location       full_label
0  20221003_120853_100162_av.png  False  120853     forarm     Right forarm
1  20221018_100146_100162_av.png   True  100146       Head             Head
2  20221002_100230_100162_av.png  False  100230  Lower Leg  Right Lower Leg
3  20221019_131930_100162_av.png  False  131930       Back             Back
4  20221003_134505_100162_av.png  False  134505      Belly            Belly
5  20220922_105623_100162_av.png  False  105623       Neck             Neck
6  20220928_103401_100162_av.png   True  103401       Head             Head
7  20221019_102925_100162_av.png   True  102925       Head             Head